In [1]:
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../dashgrid'))
sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))
from dashgrid import dgrid
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
import dash_table
import pandas as pd
import numpy as np
from dashgrid import dgrid


In [2]:
def create_dt_div(df_in=None,
                  editable_columns_in=None,title='Dash Table'):
    editable_columns = [] if editable_columns_in is None else editable_columns_in
#     datatable_id = f'{html_id}_datatable'
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
        page_action='native',
        sort_action='native',
        filter_action='none', # 'fe',
#             content_style='grow',
        style_cell_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ] + [
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
            'maxHeight':'450','overflowX': 'scroll',
        } ,
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
#         id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':['empty']})
    else:
        df = df_in.copy()
    dt.data=df.to_dict("rows")
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    child_div = html.Div([html.H4(title,style={'height':'3px'}),dt])
    return child_div

def file_store_transformer(contents):
    if contents is None or len(contents)<=0 or contents[0] is None:
        d =  None
    else:
        d = dgrid.parse_contents(contents).to_dict('rows')
    return d


In [3]:
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': 'dashed',
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
u1 = dcc.Upload(
            id='u1',
            children=html.Div(['enter a csv file']),
            # Allow multiple files to be uploaded
            multiple=False,
            style=button_style)

s1 = dcc.Store(id='s1')
s2 = dcc.Store(id='s2')

dt1 = html.Div([create_dt_div()],id='dt1')

u1_lambda = lambda value_list: [None,None] if value_list[0] is None else [file_store_transformer(value_list[0]),None]
u1_comp = dgrid.ComponentWrapper(u1,
                 input__tuples=[(u1.id,'contents')],
                 output_tuples=[(s1.id,'data'),(s2.id,'data')],
                 callback_input_transformer=u1_lambda)

dt1_lambda = lambda value_list: [None] if value_list[0] is None else [create_dt_div(pd.DataFrame(value_list[0]))]
dt1_comp = dgrid.ComponentWrapper(dt1,
                 input__tuples=[(s1.id,'data')],
                 output_tuples=[(dt1.id,'children')],
                 callback_input_transformer=dt1_lambda)


app = dash.Dash()
app.layout = html.Div([u1_comp.html,s1,s2,dt1_comp.html])
# [c.callback(app) for c in [u1_comp,dt1_comp]]
u1_comp.callback(app)
dt1_comp.callback(app)



<function dashgrid.dgrid.ComponentWrapper.callback.<locals>.execute_callback(*inputs_and_states)>

In [ ]:
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2019-07-17 17:52:45,229 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2019-07-17 17:52:49,553 - werkzeug - INFO - 127.0.0.1 - - [17/Jul/2019 17:52:49] "GET / HTTP/1.1" 200 -
2019-07-17 17:52:49,617 - werkzeug - INFO - 127.0.0.1 - - [17/Jul/2019 17:52:49] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-17 17:52:49,620 - werkzeug - INFO - 127.0.0.1 - - [17/Jul/2019 17:52:49] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-17 17:52:49,641 - werkzeug - INFO - 127.0.0.1 - - [17/Jul/2019 17:52:49] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-17 17:52:49,642 - werkzeug - INFO - 127.0.0.1 - - [17/Jul/2019 17:52:49] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563025508 HTTP/1.1" 200 -
2019-07-17 17:52:49,649 - werkzeug - INFO - 127.0.0.1 